In [4]:
!pip install openai
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.4 MB/s eta 0:00:00


In [5]:
import numpy as np
import openai
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.optimizers import Adam
import pymysql


In [6]:
db_settings = {
    "host": "zfold.one",
    "port": 3306,
    "user": "user1_db",
    "password": "P@ssword_db",
    "db": "sys",
    "charset": "utf8"
}     

In [14]:
positive_examples = []
negative_examples = []
all_texts = []

sql = ''' SELECT question, answer FROM sys.VALIDATION WHERE type='+' '''
sql2 = ''' SELECT question, answer FROM sys.VALIDATION WHERE type='-' '''
conn = pymysql.connect(**db_settings)
with conn.cursor() as cursor:
  cursor.execute(sql)
  for item in cursor.fetchall():
    positive_examples.append((item[0], item[1]))
    #print(item[0], item[1])
  cursor.execute(sql2)
  for item in cursor.fetchall():
    negative_examples.append((item[0], item[1]))

print(positive_examples)
print(negative_examples)
all_texts = [question + " " + answer for question, answer in positive_examples + negative_examples]
print(all_texts)


[('緯創軟體的主要業務是什麼？', ',緯創軟體主要從事軟體開發和技術支援服務。'), ('緯創軟體成立於哪一年？', ',緯創軟體成立於1992年。'), ('緯創軟體的總部在哪裡？', ',緯創軟體的總部位於台灣新北市。'), ('緯創軟體的員工人數大約是多少？', ',緯創軟體的員工人數大約為5000人。'), ('緯創軟體的主要客戶有哪些？', ',緯創軟體的主要客戶包括華碩、宏碁、惠普等知名品牌。'), ('緯創軟體的服務範圍包括哪些領域？', ',緯創軟體的服務範圍包括資訊技術顧問、系統整合、應用開發和技術支援。'), ('緯創軟體的股票代號是多少？', ',緯創軟體的股票代號是4953。'), ('緯創軟體在全球有多少分公司？', ',緯創軟體在全球共有約40個分公司。'), ('緯創軟體的主要市場是哪些？', ',緯創軟體的主要市場包括台灣、中國、美國、歐洲和亞洲其他地區。'), ('緯創軟體的企業文化是怎樣的？', ',緯創軟體的企業文化注重創新、團隊合作和客戶滿意度。'), ('緯創軟體的最新產品有哪些？', ',緯創軟體最新產品包括物聯網應用、人工智能解決方案等。'), ('緯創軟體的願景是什麼？', ',緯創軟體的願景是成為全球領先的軟體服務和解決方案提供商。'), ('緯創軟體的使命是什麼？', ',緯創軟體的使命是通過提供創新的軟體服務和解決方案，幫助客戶提高競爭力和實現成功。'), ('緯創軟體的核心價值觀是什麼？', ',緯創軟體的核心價值觀包括客戶至上、創新求進、團隊合作和持續改進。'), ('緯創軟體在企業社會責任方面的表現如何？', ',緯創軟體致力於履行企業社會責任，包括環境保護、員工福利和支持社區發展等方面。'), ('緯創軟體的競爭優勢是什麼？', ',緯創軟體的競爭優勢包括專業的技術團隊、豐富的行業經驗和廣泛的客戶基礎。'), ('緯創軟體的營收主要來源是什麼？', ',緯創軟體的營收主要來自軟體開發和技術支援服務。'), ('緯創軟體在研發方面的投入情況如何？', ',緯創軟體一直重視研發投入，並將其作為公司長期發展的基石。'), ('緯創軟體與哪些大學或研究機構有合作？', ',緯創軟體與多所大學和研究機構合作，例如台灣大學、清華大學等。'), ('緯創軟體的知識產權布局情況如何？', ',緯創軟體持續投資於知識產

In [15]:

vectorizer = TfidfVectorizer(max_features=128)
vectorizer.fit(all_texts)


TfidfVectorizer(max_features=128)

In [16]:

def text_to_features(examples, vectorizer):
    all_texts = [question + " " + answer for question, answer in examples]
    features = vectorizer.transform(all_texts)
    return features.toarray()


In [18]:

X_positive = text_to_features(positive_examples, vectorizer)
X_negative = text_to_features(negative_examples, vectorizer)

X_all = np.concatenate((X_positive, X_negative), axis=0)
y_all = np.concatenate((np.ones(len(positive_examples)), np.zeros(len(negative_examples))), axis=0)

# 構建神經網路模型
input_dim = X_all.shape[1]
model = Sequential([
    Dense(32, activation='relu', input_shape=(input_dim,)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 編譯模型
model.compile(optimizer=Adam(learning_rate=0.03), loss='binary_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_all, y_all, epochs=200, batch_size=4)

# 將模型的架構存儲為 JSON
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# 將模型的權重存儲為 HDF5
model.save_weights("model_weights.h5")

Epoch 1/200
23/23 [==============================] - 1s 2ms/step - loss: 0.7124 - accuracy: 0.4725
Epoch 2/200
23/23 [==============================] - 0s 2ms/step - loss: 0.5115 - accuracy: 0.8022
Epoch 3/200
23/23 [==============================] - 0s 2ms/step - loss: 0.0534 - accuracy: 1.0000
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 5/200
23/23 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 6/200
23/23 [==============================] - 0s 1ms/step - loss: 6.7046e-04 - accuracy: 1.0000
Epoch 7/200
23/23 [==============================] - 0s 1ms/step - loss: 5.0469e-04 - accuracy: 1.0000
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 3.9255e-04 - accuracy: 1.0000
Epoch 9/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1732e-04 - accuracy: 1.0000
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.6441e-04 - accura